In [1]:
from dotenv import load_dotenv
load_dotenv('./drive/MyDrive/apikeys.txt')

True

In [2]:
import json
import asyncio

In [3]:
from openai import AsyncOpenAI, OpenAI

sync_client = OpenAI()
client = AsyncOpenAI()

In [4]:
def llm_call(prompt: str,  model: str = "gpt-4o-mini") -> str:
    messages = []
    messages.append({"role": "user", "content": prompt})
    chat_completion = sync_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    return chat_completion.choices[0].message.content

In [5]:
async def llm_call_async(prompt: str,  model: str = "gpt-4o-mini") -> str:
    messages = []
    messages.append({"role": "user", "content": prompt})
    chat_completion = await client.chat.completions.create(
        model=model,
        messages=messages,
    )
    print(model,"완료")
    return chat_completion.choices[0].message.content

In [6]:
async def run_llm_parallel(prompt_list):
    tasks = [llm_call_async(prompt) for prompt in prompt_list]
    responses=[]
    for task in asyncio.as_completed(tasks):
        result = await task
        responses.append(result)
    return responses

In [7]:
user_query = "AI는 미래 일자리에 어떤 영향을 미칠까?"

In [8]:
orchestrator_prompt= f"""
다음 사용자 질문을 분석하고, 이를 3개의 관련된 하위 질문으로 분해하십시오:

다음 형식으로 응답을 제공하십시오:

{{
    "analysis": "사용자 질문에 대한 이해를 상세히 설명하고, 작성한 하위 질문들의 근거를 설명하십시오.",
    "subtasks": [
        {{
            "description": "이 하위 질문의 초점과 의도를 설명하십시오.",
            "sub_question": "질문 1"
        }},
        {{
            "description": "이 하위 질문의 초점과 의도를 설명하십시오.",
            "sub_question": "질문 2"
        }}
        // 필요에 따라 추가 하위 질문 포함
    ]
}}
최대 3개의 하위 질문을 생성하세요

사용자 질문: {user_query}"""

In [9]:
orchestrator_prompt

'\n다음 사용자 질문을 분석하고, 이를 3개의 관련된 하위 질문으로 분해하십시오:\n\n다음 형식으로 응답을 제공하십시오:\n\n{\n    "analysis": "사용자 질문에 대한 이해를 상세히 설명하고, 작성한 하위 질문들의 근거를 설명하십시오.",\n    "subtasks": [\n        {\n            "description": "이 하위 질문의 초점과 의도를 설명하십시오.",\n            "sub_question": "질문 1"\n        },\n        {\n            "description": "이 하위 질문의 초점과 의도를 설명하십시오.",\n            "sub_question": "질문 2"\n        }\n        // 필요에 따라 추가 하위 질문 포함\n    ]\n}\n최대 3개의 하위 질문을 생성하세요\n\n사용자 질문: AI는 미래 일자리에 어떤 영향을 미칠까?'

In [10]:
orchestrator_response = llm_call(orchestrator_prompt, model='gpt-4o')

In [11]:
orchestrator_response

'```json\n{\n    "analysis": "사용자 질문은 AI가 미래의 고용 환경과 직업 시장에 미칠 영향을 이해하고자 하는 것에 초점을 맞추고 있습니다. AI의 발전은 직업의 성격을 변화시키고 새로운 직업을 창출하며 일부 직업을 대체할 수 있기 때문에 이에 대한 관심이 높아지고 있습니다. 하위 질문들은 AI가 일자리에 미치는 영향의 다양한 측면을 살펴보도록 구성되었습니다.",\n    "subtasks": [\n        {\n            "description": "이 하위 질문은 AI가 어떤 유형의 일자리를 대체하거나 감소시킬 가능성이 있는지를 이해하는 데 초점을 맞춥니다. AI는 특히 반복적이고 예측 가능한 작업을 자동화하는 데 강점이 있기 때문에, 이러한 변화가 직업 시장에 어떤 영향을 미칠지 질문합니다.",\n            "sub_question": "AI는 어떤 직업을 대체할 것인가?"\n        },\n        {\n            "description": "이 하위 질문은 AI의 발달로 인해 새롭게 생성될 수 있는 일자리의 유형과 이러한 직업들이 요구하는 기술 및 자격에 대해 탐구하는 것입니다. 이를 통해 개인과 교육 기관이 미래의 일자리 요구에 대비할 수 있도록 합니다.",\n            "sub_question": "AI로 인해 새롭게 창출될 일자리는 어떤 것들이 있을까?"\n        },\n        {\n            "description": "이 하위 질문은 AI의 도입이 노동 시장의 구조적 변화를 유발할 것인가에 대해 질문합니다. 예를 들어 AI로 인한 실업률 변화나 일자리의 성격 변화 등이 어떻게 노동 시장 동향에 영향을 미칠 수 있는지를 살펴봅니다.",\n            "sub_question": "AI는 노동 시장의 구조에 어떤 변화를 가져올 것인가?"\n        }\n    ]\n}\n```'

In [13]:
response_json=json.loads(orchestrator_response.replace('```json', '').replace('```',''))